__[Problem 1J](http://rosalind.info/problems/ba1j/)__

>  __Frequent Words with Mismatches and Reverse Complements Problem__ - `FreqWordsMismatchesRevComplements( text, k, d )`

*Objective:* Find most frequent $k$-mers (with mismatches and reverse complements) in a DNA string

*  Input: a DNA string (*Text*), and integers $k, d$
*  Output: all most frequent $k$-mers *Pattern* maximizing the sum $\textit{Count}_d(\textit{Text, Pattern}) + \textit{Count}_d(\textit{Text}, \overline{\textit{Pattern}})$ over all possible $k$-mers

Def:
*  given strings *Text*, *Pattern*, and integer $d$, we define $\textit{Count}_d(\textit{Text, Pattern})$ as the total number of occurences of *Pattern* in *Text* with a most $d$ mismatches,
*  a __most frequent $k$-mer with up to $d$ mismatches__ is *Text* is a string *Pattern* maximizing $\textit{Count}_d(\textit{Text, Pattern})$ among all $k$-mers.

In [1]:
DNAdict = {'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

def ReverseComplement( Pattern ):
    RC = str()
    for i in reversed(Pattern):
        if i in DNAdict.keys():
            RC += DNAdict[i]
        else:
            RC = 'Wrong DNA'
            break
    return RC

In [2]:
def HammingDistance(String1, String2):
    if len(String1) != len(String2):
        distance = 'Error: Lengths do not match.'
    else:
        distance = 0
        for i in range(0,len(String1)):
            if String1[i] != String2[i]:
                distance += 1
    return distance

In [3]:
def d_Nbhd(Pattern, d):
    
    # Trivial cases
    if d == 0:
        return Pattern
    if len(Pattern) == 1:
        return ['A', 'C', 'G', 'T']
    
    Nbhd = set()
    Suffix_Pattern = Pattern[1:]
    Suffix_Nbhd = d_Nbhd(Suffix_Pattern, d)
    
    for text in Suffix_Nbhd:
        if HammingDistance(Suffix_Pattern, text) < d:
            for x in ['A', 'C', 'G', 'T']:
                if (x + text) not in Nbhd:
                    Nbhd.add(x + text)
        else:
            if (Pattern[0] + text) not in Nbhd:
                Nbhd.add(Pattern[0] + text)
        
    return Nbhd

In [4]:
def FreqWordsMismatchesRevComplements( Text, k, d ):
    
    # Initialize helper values, lists
    noChar = len(Text)
    pttn_Nbhd = set()
    
    # Loop 1: Get all possible k-mers with <= d mismatches from what observed
    for i in range(0, noChar-k+1):
        pattern = Text[i:i+k]
        # Small fix for if d=0, pattern is a set of length 1, need to convert to list
        if d == 0:
            pttn_Nbhd = pttn_Nbhd.union([d_Nbhd(pattern, d)])
        else:
            pttn_Nbhd = pttn_Nbhd.union(d_Nbhd(pattern, d))
    
    # Given all d-nbhd patterns, find each Reverse Complement
    pttn_Nbhd = list(pttn_Nbhd)
    pttn_RC = []
    for pttn in pttn_Nbhd:
        RevComp = ReverseComplement(pttn)
        pttn_RC.append(RevComp)
     
    # Loop 2: Get Frequency of each of those k-mers from from Loop 1
    count_arr_Pttn = []
    for pttn in pttn_Nbhd:
        # Initialize count of current pattern
        count = 0
        # Loop 2.1: Consider each pattern string in Text
        for i in range(0, noChar-k+1):
            # Get current string
            stringText = Text[i:i+k]
            # Check for if Hamming distance <= d
            if HammingDistance(pttn, stringText) <= d:
                count += 1
        # Done counting, add to record
        count_arr_Pttn.append(count)
        
    # Loop 3: Get Frequency of each of the respective Reverse Complement from from Loop 1
    count_arr_RC = []
    for pttn in pttn_RC:
        # Initialize count of current pattern
        count = 0
        # Loop 2.1: Consider each pattern string in Text
        for i in range(0, noChar-k+1):
            # Get current string
            stringText = Text[i:i+k]
            # Check for if Hamming distance <= d
            if HammingDistance(pttn, stringText) <= d:
                count += 1
        # Done counting, add to record
        count_arr_RC.append(count)
    
    # Add Count(pattern) + Count(RevComp(pattern))
    count_arr_All = []
    for i in range(0,len(count_arr_Pttn)):
        count_arr_All.append(count_arr_Pttn[i] + count_arr_RC[i])
    max_count = max(count_arr_All)
    indices = [i for i, x in enumerate(count_arr_All) if x == max_count]
    
    # Get the most frequent k-mer patterns
    most_freq = []
    for i in range(0,len(indices)):
        most_freq.append(pttn_Nbhd[indices[i]])
        most_freq.append(pttn_RC[indices[i]])
    
    return set(most_freq)

In [5]:
Text = 'ACGTTGCATGTCGCATGATGCATGAGAGCT'
print('Sample test: ' + str(FreqWordsMismatchesRevComplements(Text,4,1)))

Sample test: {'ATGT', 'ACAT'}


In [6]:
Text = 'AAAAAAAAAA'
print('Test 1: ' + str(FreqWordsMismatchesRevComplements(Text,2,1)))

Test 1: {'AT', 'TA'}


In [7]:
Text = 'AGTCAGTC'
print('Test 2: ' + str(FreqWordsMismatchesRevComplements(Text,4,2)))

Test 2: {'AATT', 'GGCC'}


In [8]:
Text = 'AATTAATTGGTAGGTAGGTA'
print('Test 3: ' + str(FreqWordsMismatchesRevComplements(Text,4,0)))

Test 3: {'AATT'}


In [9]:
Text = 'ATA'
print('Test 4: ' + str(FreqWordsMismatchesRevComplements(Text,3,1)))

Test 4: {'TGT', 'TAG', 'TAA', 'ATG', 'TCT', 'ATA', 'TAT', 'CTA', 'AAA', 'AAT', 'CAT', 'TAC', 'GTA', 'ACA', 'ATC', 'TTA', 'GAT', 'TTT', 'AGA', 'ATT'}


In [10]:
Text = 'AAT'
print('Test 5: ' + str(FreqWordsMismatchesRevComplements(Text,3,0)))

Test 5: {'AAT', 'ATT'}


In [11]:
Text = 'TAGCG'
print('Test 6: ' + str(FreqWordsMismatchesRevComplements(Text,2,1)))

Test 6: {'CA', 'CC', 'GG', 'TG'}


In [12]:
Text = 'CTTGCCGGCGCCGATTATACGATCGCGGCCGCTTGCCTTCTTTATAATGCATCGGCGCCGCGATCTTGCTATATACGTACGCTTCGCTTGCATCTTGCGCGCATTACGTACTTATCGATTACTTATCTTCGATGCCGGCCGGCATATGCCGCTTTAGCATCGATCGATCGTACTTTACGCGTATAGCCGCTTCGCTTGCCGTACGCGATGCTAGCATATGCTAGCGCTAATTACTTAT'
print('Extra Test: ' + str(FreqWordsMismatchesRevComplements(Text,9,3)))

Extra Test: {'AGCGGCGCT', 'AGCGCCGCT'}


<br/>

[Rosalind final test:](http://rosalind.info/problems/ba1i/)

In [13]:
Text = 'GCGGTTACGATAGAAGCCTACACGCGACGAATTAATCTAGCGTTTCTACACGCGACGAGCGTTTCTAGCGTTTCTGCGGTTACGAGCGTTTCTAATTAATCTGCGGTTACGACACGCGACGGCGGTTACGATAGAAGCCTGCGGTTACGAATTAATCTAATTAATCTGCGGTTACGGCGGTTACGATAGAAGCCTATAGAAGCCTGCGGTTACGAGCGTTTCTACACGCGACGAGCGTTTCTAATTAATCTATAGAAGCCTGCGGTTACGAGCGTTTCTATAGAAGCCTATAGAAGCCTAATTAATCTAGCGTTTCTATAGAAGCCTATAGAAGCCTAATTAATCTGCGGTTACGGCGGTTACGGCGGTTACGGCGGTTACGATAGAAGCCTATAGAAGCCTAATTAATCTATAGAAGCCTAATTAATCTGCGGTTACGGCGGTTACGGCGGTTACGAGCGTTTCTAGCGTTTCTGCGGTTACGAATTAATCTGCGGTTACGAATTAATCTAGCGTTTCTAGCGTTTCTACACGCGACGAGCGTTTCTAGCGTTTCTAATTAATCTACACGCGACGGCGGTTACGAGCGTTTCTGCGGTTACGAATTAATCTAGCGTTTCTGCGGTTACGGCGGTTACGGCGGTTACGAGCGTTTCTAATTAATCTATAGAAGCCTAATTAATCTATAGAAGCCTACACGCGACGAGCGTTTCTAGCGTTTCTAATTAATCTGCGGTTACGATAGAAGCCTGCGGTTACGAGCGTTTCTATAGAAGCCTAATTAATCTAATTAATCTATAGAAGCCTGCGGTTACGAGCGTTTCTAATTAATCTGCGGTTACGAATTAATCTACACGCGACGACACGCGACGGCGGTTACGACACGCGACG'
answer = FreqWordsMismatchesRevComplements(Text,6,3)
print('Final test: ' + str(answer))

Final test: {'TTTAAA'}
